<a href="https://colab.research.google.com/github/shoeb370/Audio-Classification/blob/main/optuna_final_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
poly_files = sorted(glob(r'/content/drive/MyDrive/jayesh_work/Dissertation/polysonmography_new/mesa-sleep-*.csv')) #Need to change /content/drive/MyDrive/jayesh_work/Dissertation/polysonmography_new
poly_files
# /content/drive/MyDrive/jayesh_work/Dissertation/mesa_new

In [ ]:
act_files = sorted(glob(r'/content/drive/MyDrive/jayesh_work/Dissertation/mesa_new/mesa-sleep-*.csv')) #Need to change /content/drive/MyDrive/jayesh_work/Dissertation/mesa_new
act_files

In [ ]:
poly_df = pd.DataFrame({"poly_filename":poly_files})
poly_df.head()

In [ ]:
poly_df.poly_filename.iloc[0]


In [ ]:
new_files = [itm[0] for itm in poly_df.poly_filename.str.findall(r"(?<=polysonmography_new/)(.*?)(?=-rpoint)") if len(itm)>0]
# new_files

In [ ]:
poly_df["match_filename"]= new_files
poly_df.shape

In [ ]:
poly_df.head()

In [ ]:
act_df = pd.DataFrame({"act_filename":act_files})
# act_df
# act_files[0]

In [ ]:
act_new_files = [itm[0] for itm in act_df.act_filename.str.findall(r"(?<=mesa_new/)(.*?)(?=.csv)") if len(itm)>0]
# act_new_files
# act_new_files[0]

In [ ]:
act_df["match_filename"]= act_new_files
# act_df.shape
# act_df.head()


In [ ]:
temp_final_df = poly_df.merge(act_df, how='inner')
# temp_final_df.shape
# temp_final_df.head()

In [ ]:
temp_final_df.poly_filename.iloc[3]

In [ ]:
X = pd.DataFrame()
Y = pd.DataFrame()

# for i in range(5):
for i in range(len(temp_final_df)):
    
    temp_df1_act = pd.read_csv(temp_final_df.act_filename.iloc[i])
    temp_df1_poly = pd.read_csv(temp_final_df.poly_filename.iloc[i])
    temp_df1_poly["stage"] = temp_df1_poly["stage"].fillna(0)
    df1_truncated= temp_df1_act.tail(len(temp_df1_poly))
    # df1_truncated= df1_truncated[["offwrist", "activity","whitelight","redlight","greenlight","bluelight"]].fillna(0)
    df1_truncated= df1_truncated[["offwrist", "activity"]].fillna(0)
    # X = X.append(df1_truncated[["offwrist", "activity","whitelight","redlight","greenlight","bluelight"]])
    X = X.append(df1_truncated[["offwrist", "activity"]])
    Y = Y.append(temp_df1_poly[["stage"]])


In [ ]:
X.shape, Y.shape

In [ ]:
# X = X[["offwrist", "activity"]] #To comment to get all redlight, whitelight
X['activity'] = X['activity'].astype(np.float64)
X['offwrist'] = X['offwrist'].astype(np.float64)
Y['stage'] = Y['stage'].astype(np.float64)

X.activity.dtypes, X.offwrist.dtypes, Y.stage.dtypes

In [ ]:
Y2 = Y['stage'].apply(lambda x: 0.0 if x == 0.0 else 1.0)

In [ ]:
X2 = X
# X2 = X.head(5000) #to increase the length of the data
# Y2 = Y2.head(5000) #to increase the length of the data

In [ ]:
!pip install optuna

In [ ]:
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
import optuna
def objective(trial):
      # iris = sklearn.datasets.load_iris()
      n_estimators = trial.suggest_int('n_estimators', 200, 2000,10)
      max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))
      clf = sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
      return sklearn.model_selection.cross_val_score(clf, X2, Y2,
           n_jobs=-1, cv=3).mean()

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
print("random forest best parameter: ",study.best_params)

In [ ]:
rf_n_estimators = study.best_params['n_estimators']
rf_max_depth = study.best_params['max_depth']

In [ ]:
#Build RF model
rf_clf = RandomForestClassifier(n_estimators = rf_n_estimators, max_depth= rf_max_depth)
rf_clf.fit(X2, Y2)
rf_accuracy = rf_clf.score(X2, Y2)
print(rf_accuracy)

In [ ]:
#Save the model of the Random forest
import numpy as np
import pickle
pickle.dump(rf_clf, open('rf_model.pkl', 'wb'))

In [ ]:
# load the saved model of RF
#I have taken x2[78] data which output should be 0
pickled_model = pickle.load(open('rf_model.pkl', 'rb'))
print(f"the model output is:{pickled_model.predict([[21,0]])} \n","-"*50)



In [ ]:
#SVC Best parameter
import optuna
import sklearn.svm
def objective_svc(trial):
    c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
    ## suggest_float(name, low, high, *, step=None, log=False)
    ## Suggest a value for the floating point parameter.
    clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf,X2,Y2, n_jobs=-1, cv=3).mean() # cv= cross validation = 3. precision,recall, f1 score

In [ ]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective_svc, n_trials=100)

# trial = study.best_trial

# print('Accuracy: {}'.format(trial.value))
# print("Best hyperparameters: {}".format(trial.params))

In [ ]:
# print("SVC's  best parameter: ",study.best_params)
# c_best = study.best_params['svc_c']

In [ ]:
# #building SVM - classification model for best parameters
# svm_clf = sklearn.svm.SVC(C=c_best, gamma='auto')
# svm_clf.fit(X2,Y2)
# svc_accuracy = svm_clf.score(X2,Y2)
# print(svc_accuracy)


In [ ]:
# # Saving svm model and predicting
# import numpy as np
# import pickle
# pickle.dump(svm_clf, open('svm_clf_model.pkl', 'wb'))

In [ ]:
# # load the saved model of SVM - C
# pickled_model = pickle.load(open('svm_clf_model.pkl', 'rb'))
# print(f"the model output is:{pickled_model.predict([[11,0]])} \n","-"*50)


In [ ]:
#XGB OPTUNA
import xgboost as xgb
def objective_xgb(trial):
    # (data, target) = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(X2, Y2, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [ ]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgb, n_trials=100, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():

        print("    {}: {}".format(key, value))

In [ ]:
print("XGB  best parameter: ",study.best_params)


In [ ]:
# XGBClassifier
xgb_clf = xgb.XGBClassifier(params = study.best_params )
xgb_clf.fit(X2,Y2)
xgb_accuracy = xgb_clf.score(X2,Y2)
print(xgb_accuracy)

In [ ]:
#Save the model of XGB
import numpy as np
import pickle
pickle.dump(xgb_clf, open('xgb_clf_model.pkl', 'wb'))

In [ ]:
offwrist_data = eval(input("Enter the offwrist value:"))
activity_data = eval(input("Enter the activity value:"))
list_off = [offwrist_data]; list_act = [activity_data]
x_test_xgb = pd.DataFrame({"offwrist":[offwrist_data],"activity":[activity_data]})
x_test_xgb[:1]
xgb_clf.predict(x_test_xgb)

In [ ]:
# load the saved model of XGB -Clf
pickled_model = pickle.load(open('xgb_clf_model.pkl', 'rb'))
print(f"the model output is:{pickled_model.predict(x_test_xgb)} \n","-"*50)


In [ ]:
# rf_accuracy, svc_accuracy, xgb_accuracy

In [ ]:
#  y_plot = [x for x in range(0.0,1.1,0.1)]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

def objective_dt(trial):
  # max_depth = trial.suggest_int('max_depth', 2, 100 ) #X2.shape[1]
  max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))
  min_samples_split = trial.suggest_int('min_samples_split', 2, 1000)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 1000)
  max_features = trial.suggest_categorical("max_features", ["auto","log2","sqrt",None])
  clf = DecisionTreeClassifier(max_depth = max_depth, min_samples_split = min_samples_split,min_samples_leaf = min_samples_leaf,max_features = max_features)
  
  return sklearn.model_selection.cross_val_score(clf, X2, Y2,
           n_jobs=-1, cv=3).mean()

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_dt, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
print("Decision Tree's  best parameter: ",study.best_params)


In [ ]:
dt_clf = DecisionTreeClassifier(max_depth = study.best_params['max_depth'],
                             min_samples_split = study.best_params['min_samples_split'],
                             min_samples_leaf = study.best_params['min_samples_leaf'],
                             max_features = study.best_params['max_features'])

In [ ]:
dt_clf.fit(X2,Y2)

In [ ]:
dt_clf.score(X2,Y2)

In [ ]:
#Save the model

# Saving Decision Tree  model and predicting
import numpy as np
import pickle
pickle.dump(dt_clf, open('dt_clf_model.pkl', 'wb'))


In [ ]:
# load the saved model of Decision Tree classifier model
pickled_model = pickle.load(open('dt_clf_model.pkl', 'rb'))
print(f"the model output is:{pickled_model.predict([[11,0]])} \n","-"*50)
